# **Importing Libraries**

In [ ]:
import os
import datetime
import click
import numpy as np
import tqdm

from denoise.utils import load_images, write_log
from denoise.losses import wasserstein_loss, perceptual_loss
from denoise.model import generator_model, discriminator_model, generator_containing_discriminator_multiple_outputs

from keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

from numpy import load

In [ ]:
# This block of code is to adapt with new tensorflow verison(2.6)

import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)
#tf.data.experimental.enable.debug_mode(True)

# **Storing Directory Settings**

In [ ]:
# Set the directory where the weights will be saved
BASE_DIR = './weights/'

# Weight saving function
def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join(BASE_DIR, '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)

# **Parameters**

In [ ]:
n_images = 1500
batch_size = 15
log_dir = True
epoch_num = 30000
critic_updates=5

# **Load Data**

In [ ]:
data = load_images('../input/dataset/Train/', 1500)

In [ ]:
x_train, y_train = data['A'], data['B']

# **Train**

In [ ]:
# Load models
g = generator_model()
d = discriminator_model()
d_on_g = generator_containing_discriminator_multiple_outputs(g, d)

# Set the optimizers
d_opt = Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_on_g_opt = Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# Set the compilers
d.trainable = True
d.compile(optimizer=d_opt, loss=wasserstein_loss)
d.trainable = False
loss = [perceptual_loss, wasserstein_loss]
loss_weights = [100, 1]
d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
d.trainable = True

output_true_batch, output_false_batch = np.ones((batch_size, 1)), -np.ones((batch_size, 1))

log_path = './logs'
tensorboard_callback = TensorBoard(log_path)

# Load saved weights when starting again from a trained point
#g.load_weights('../input/dataset/generator_120_226.h5')
#d.load_weights('../input/dataset/discriminator_120.h5')
    
for epoch in tqdm.tqdm(range(epoch_num)):
    permutated_indexes = np.random.permutation(x_train.shape[0])

    d_losses = []
    d_on_g_losses = []
    for index in range(int(x_train.shape[0] / batch_size)):
        batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
        image_blur_batch = x_train[batch_indexes]
        image_full_batch = y_train[batch_indexes]

        generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

        for _ in range(critic_updates):
            d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
            d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
            d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            d_losses.append(d_loss)
                
        #print("d_loss: ", d_loss)
        #print("Batch :", index+1)
        d.trainable = False

        d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])
        d_on_g_losses.append(d_on_g_loss)

        d.trainable = True

    # write_log(tensorboard_callback, ['g_loss', 'd_on_g_loss'], [np.mean(d_losses), np.mean(d_on_g_losses)], epoch_num)
    print(np.mean(d_losses), np.mean(d_on_g_losses))
    with open('./log(1500).txt', 'a+') as f:
        f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

    if epoch%10==0:
        save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))